# Homework II: Artificial Intelligence Topics
## Viterbi algorithm in Hiden Markov Model

by Isaías Huerta Vargas

Universidad de Concepción

Computer Science Department

## Goal

The work consists of implementing the **viterbi algorithm** to find the most likely path for an agent in a 2D grid, given a sequence of observations.

## Setup

In [328]:
import numpy as np
from termcolor import colored

### States

In [329]:
"""
Map:

   1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16
  _______________________________________________
1 |0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0|
2 |1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1|
3 |1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0|
4 |0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0|
  ------------------------------------------------
"""
states = (
        [1,1],[1,2],[1,3],[1,4],      [1,6],[1,7],[1,8],[1,9],[1,10],       [1,12],[1,13],[1,14],       [1,16],
                    [2,3],[2,4],      [2,6],            [2,9],       [2,11],       [2,13],
              [3,2],[3,3],[3,4],      [3,6],            [3,9],[3,10],[3,11],[3,12],[3,13],              [3,16],
        [4,1],[4,2],      [4,4],[4,5],[4,6],      [4,8],[4,9],[4,10],[4,11],       [4,13],[4,14],[4,15],[4,16],
)

n_states = len(states) # 42

### Observations
**Nomenclature:** [N, E, S, W], 1 if there is wall, 0 if not.

In [330]:
observation_space = (
    [1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1], # One wall
    [1,0,1,0],[0,1,0,1],                     # Two parallel walls
    [1,1,0,0],[0,1,1,0],[0,0,1,1],[1,0,0,1], # Two diagonal walls
    [0,1,1,1],[1,0,1,1],[1,1,0,1],[1,1,1,0], # Three walls
    [0,0,0,0],[1,1,1,1]                      # Open and closed
)

n_obs = len(observation_space) # 16
        

### Transition and Emission matrix
Creating:
* A(n_states x n_states): **transition matrix**. $A_{ij}$ stores the transition probability of transiting from state $s_{i}$ to state $s_{j}$ 
* B(n_states x n_obs): **emission matrix**. $B_{ij}$ stores the probability of observing $o_{j}$ from state $ s_{i}$

In [331]:
A = np.zeros((n_states, n_states))
B = np.zeros((n_states, n_obs))
def createMatrix(epsilon):
    for num, pair in enumerate(states):
        i,j = pair
        aux = np.zeros(n_states)
        obs_aux = [1,1,1,1]
        if [i-1,j] in states: #no North wall
            aux[states.index([i-1,j])]=1
            obs_aux[0] = 0
        if [i,j+1] in states: #no East wall
            aux[states.index([i,j+1])]=1
            obs_aux[1] = 0
        if [i+1,j] in states: #no South wall
            aux[states.index([i+1,j])]=1
            obs_aux[2] = 0
        if [i,j-1] in states: #no West wall
            aux[states.index([i,j-1])]=1
            obs_aux[3] = 0
        if sum(aux) > 0:
            aux = [float(i)/sum(aux) for i in aux] # normalizing

        A[num,:] = aux
        for o in observation_space:
            diff = 0
            for real, sensor in zip(o, obs_aux):
                if real != sensor: # counting differences
                    diff = diff + 1
            B[num][observation_space.index(o)] = ((1 - epsilon) ** (4-diff)) * (epsilon ** diff) # sensor model

        for n,arr in enumerate(B):
            if sum(arr) > 0:
                B[n] = [float(i)/sum(arr) for i in arr] # normalizing

### Initial Probabilities
All states have equal probability at the beginning

In [332]:
pi = np.array([1/n_states]*n_states)

## Viterbi
Returns the most likely state sequence given observed sequence x. Implementation based on [Wikipedia's pseudocode](https://en.wikipedia.org/wiki/Viterbi_algorithm#Pseudocode)

In [333]:
def viterbi(obs, epsilon):
    createMatrix(epsilon)
    T = len(obs) # observation size
    T1 = np.zeros((T, n_states)) # probability of the most likely path so far
    T2 = np.zeros((T, n_states))
    T1[0] = pi*B[:, observation_space.index(obs[0])] # first observation probability
    for i in range(1, T): # for each observation
        for j in range(n_states): 
            T1[i,j] = np.max(T1[i-1] * A[:,j] * B[j, observation_space.index(obs[i])])
            T2[i,j] = np.argmax(T1[i-1] * A[:,j] * B[j, observation_space.index(obs[i])])
    
    states_seq = np.zeros(T, dtype=np.int) # states path
    states_seq_prob = [0]*T                # states path probability
    states_seq[-1] = np.argmax(T1[-1])
    states_seq_prob[-1] = np.max(T1[-1])
    for i in range(T-2, -1, -1): # backtrack
        states_seq[i] = T2[i+1, states_seq[i+1]]
        states_seq_prob[i] = T1[i+1, states_seq[i+1]]
    
    wumpus_prob = T1[-1, states.index([1,10])]/sum(T1[-1]) #wumpus
    gold_prob = T1[-1, states.index([3,16])]/sum(T1[-1]) #gold
    return wumpus_prob, gold_prob, states_seq, states_seq_prob

### Graphical map

* Grey: states
* Black: walls
* Red: wumpus
* Yellow: gold
* Green: path expected


In [334]:
def graph(path):
    for i in range(1,5):
        for j in range(1,17):
            if [i,j] == [3,16]:
                print(colored(u"\u2588","yellow")," ", end='')
            elif [i,j] == [1,10]:
                print(colored(u"\u2588","red")," ", end='')
            elif [i,j] in path:
                print(colored(u"\u2588","green")," ", end='')
            elif [i,j] not in states:
                print(u"\u2588"," ", end='')
            else:
                print(colored(u"\u2588", "white")," ", end='')
        print()
        print()

# Tests and Answers

There are **eight** experiments, 4 observations * 2 epsilon, where for each one is given:
* The expected path and associated probability 
* Probability that the robot finishes in the wumpus or in the gold


## E1 and Epsilon = 0.2

In [335]:
#            NS        NS        NS        NWE
sensor = [[1,0,1,0],[1,0,1,0],[1,0,1,0],[1,1,0,1]]

wumpus, gold, path, prob = viterbi(sensor, 0.2)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 1]]
Expected path: [[1, 7], [1, 8], [1, 7], [1, 6]]
Associated probability: [0.001997287619047619, 0.0004090445043809523, 2.094307862430476e-05, 2.094307862430476e-05]
Wumpus probability: 0.016028320874253085
Gold probability: 0.09616992524551848


## E1 and Epsilon = 0.05

In [336]:
#            NS        NS        NS        NWE
sensor = [[1,0,1,0],[1,0,1,0],[1,0,1,0],[1,1,0,1]]

wumpus, gold, path, prob = viterbi(sensor, 0.05)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 0, 1]]
Expected path: [[1, 7], [1, 8], [1, 7], [1, 6]]
Associated probability: [0.00789786227725075, 0.0032164290932299856, 6.894214734520149e-05, 6.894214734520149e-05]
Wumpus probability: 0.0015690212732910266
Gold probability: 0.044717106288794245


## E2 and Epsilon = 0.2

In [337]:
#            NS        NS        NSE
sensor = [[1,0,1,0],[1,0,1,0],[1,1,1,0]]

wumpus, gold, path, prob = viterbi(sensor, 0.2)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 1, 0]]
Expected path: [[1, 8], [1, 7], [1, 8]]
Associated probability: [0.0019972876190476187, 0.00010226112609523811, 0.00010226112609523811]
Wumpus probability: 0.08629698297657175
Gold probability: 0.002022585538513401


## E2 and Epsilon = 0.05

In [338]:
#            NS        NS        NSE
sensor = [[1,0,1,0],[1,0,1,0],[1,1,1,0]]

wumpus, gold, path, prob = viterbi(sensor, 0.05)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[1, 0, 1, 0], [1, 0, 1, 0], [1, 1, 1, 0]]
Expected path: [[1, 7], [1, 8], [1, 7]]
Associated probability: [0.00789786227725075, 0.00016928574174894666, 0.00016928574174894666]
Wumpus probability: 0.11191750743912617
Gold probability: 2.4475326018178933e-05


## E3 and Epsilon = 0.2

In [339]:
#            WE         WE       NS         NS
sensor = [[0,1,0,1],[0,1,0,1],[1,0,1,0],[1,0,1,0]]

wumpus, gold, path, prob = viterbi(sensor, 0.2)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[0, 1, 0, 1], [0, 1, 0, 1], [1, 0, 1, 0], [1, 0, 1, 0]]
Expected path: [[2, 6], [3, 6], [4, 6], [4, 5]]
Associated probability: [0.001997287619047619, 2.5565281523809534e-05, 5.235769656076193e-06, 5.235769656076193e-06]
Wumpus probability: 0.022734778494098454
Gold probability: 9.990869455414358e-05


## E3 and Epsilon = 0.05

In [340]:
#            WE         WE       NS         NS
sensor = [[0,1,0,1],[0,1,0,1],[1,0,1,0],[1,0,1,0]]

wumpus, gold, path, prob = viterbi(sensor, 0.05)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[0, 1, 0, 1], [0, 1, 0, 1], [1, 0, 1, 0], [1, 0, 1, 0]]
Expected path: [[2, 6], [3, 6], [4, 6], [4, 5]]
Associated probability: [0.00789786227725075, 8.90977588152351e-06, 3.6285340708000804e-06, 3.6285340708000804e-06]
Wumpus probability: 0.00704095241418758
Gold probability: 1.2796049699080738e-08


## E4 and Epsilon = 0.2

In [341]:
#            E          E         E        NSW
sensor = [[0,1,0,0],[0,1,0,0],[0,1,0,0],[1,0,1,1]]

wumpus, gold, path, prob = viterbi(sensor, 0.2)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 1, 1]]
Expected path: [[3, 4], [2, 4], [3, 4], [4, 4]]
Associated probability: [0.0013315250793650792, 0.00018179755750264548, 6.205356629423633e-06, 6.205356629423633e-06]
Wumpus probability: 0.0001267400350647431
Gold probability: 0.002281320631165374


## E4 and Epsilon = 0.05

In [342]:
#            E          E         E        NSW
sensor = [[0,1,0,0],[0,1,0,0],[0,1,0,0],[1,0,1,1]]

wumpus, gold, path, prob = viterbi(sensor, 0.05)
path = list(map(lambda s: states[s], path))
graph(path)
print("Sensor input:", sensor)
print("Expected path:", path)
print("Associated probability:", prob)
print("Wumpus probability:", wumpus)
print("Gold probability:", gold)

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

█  █  █  █  █  █  █  █  █  █  █  █  █  █  █  █  

Sensor input: [[0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 1, 1]]
Expected path: [[3, 4], [2, 4], [3, 4], [4, 4]]
Associated probability: [0.0052652415181671675, 0.0014295240414355496, 2.0427302917096747e-05, 2.0427302917096747e-05]
Wumpus probability: 2.492463130585615e-08
Gold probability: 2.1310559766506973e-06
